1. 최근접 이웃 집합 U를 계산 


2. 식 1로 생성한 num_item_rec_top M 개의 추천 결과를 한 줄로 출력

U에서, 점수가 높은 item을 한줄로 출력

    * 추천 결과의 각 아이템 id는 공백으로 구분되어야함
    * 사용자에게 추천할 수 있는 아이템 집합 i가 아닌, 최근접 이웃 u가 평가한 아이템 집합 i' 임

1. make data frame
    input list, put in dict, trans data frame
2. transform to pivot table
    df.pivot_table('rating',index='user_id',columns='title').fillna(0)
3. calculate cosine similiarity
4. find similiar user

예측 평점 구하기.

예측 평점 =  (item 유사도 * 평점) / (item 유사도의 합)

# 예측 평점 구하기
https://wooono.tistory.com/135

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import sys


c:\Python39\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


데이터 입력

In [1]:
num_sim_user_top_N = int(input())
num_item_rec_top_M = int(input())

num_users = int(input())
num_items = int(input())
num_rows = int(input())
num_row = []

rating_data={}
items = set()
for _ in range(num_rows):
  user,item,rating = input().split(' ')
  user = int(user)
  item = int(item)
  rating = float(rating)

  if user not in rating_data:
    rating_data[user]={}
  rating_data[user][item]=rating

  items.add(item)

In [3]:
print(rating_data)
print(items)

{1: {1: 1.0, 2: 2.0, 5: 1.2}, 2: {2: 1.5, 3: 3.0}, 3: {1: 2.2, 2: 6.2, 7: 1.5}, 4: {6: 1.2, 3: 1.5, 1: 3.1, 2: 4.0}, 5: {4: 8.2, 2: 6.5, 7: 8.0}}
{1, 2, 3, 4, 5, 6, 7}


In [4]:
# 사용자 별 평균 평점 계산
user_average_rating = dict() # {사용자 ID : 평균 평점} 형태로 저장한다.
for user, item_ratings in rating_data.items():
  user_average_rating[user] = sum(item_ratings.values()) / len(item_ratings)

print(user_average_rating)

{1: 1.4000000000000001, 2: 2.25, 3: 3.3000000000000003, 4: 2.45, 5: 7.566666666666666}


In [5]:
num_reco_users = int(input())



In [15]:
user_rating = rating_data[1]
user_rating[1]

1.0

In [ ]:
for _ in range(num_reco_users):
  query_user = int(input()) # 추천할 사용자 id
  user_sim = dict() # {사용자 id:유사도} 형태로 저장한다.

  # 사용자 별 유사도 계산
  for user in rating_data.keys() - {query_user}: # 유저들 중에서, query_user에 해당하는 사람을 제외한다. 딕셔너리를 통해 제외
    user_ratings = rating_data[user]
    query_user_ratings = rating_data[query_user]
    intersection_items = rating_data[query_user].keys() & rating_data[user].keys() # query유저와 다른 사용자에대해서 겹치는 아이템 판별

    # 분자
    numerator = sum(user_ratings[item]*query_user_ratings[item] for item in intersection_items)
    # 분모
    denominator = sum(rating **2 for rating in user_ratings.values()) **0.5
    denominator *= sum(rating **2 for rating in query_user_ratings.values()) ** 0.5

    # 유사도 계산해서 저장 (계산되지 않는 경우 0으로 정의)
    user_sim[user] = numerator / denominator if denominator > 0.0 else 0.0

    # 유사도 기준으로 정렬. [(사용자 ID,유사도),...] 형태의 list of tuple로 저장
    nearest = sorted(user_sim.items(),key=lambda x:x[1],reverse=True)

    predicted_rating = list() #예측 평점 (높은 순서에 따라 추천됨)

  # 이미 평가한 정보가 있으면 예측 평점을 계산하지 않는다.

  for item in items - rating_data[query_user].keys(): # for 문의 조건식에서 필터링하네
    sum_k=0 # 평균화된 factor k 계산을 위한 sum임
    sum_u=0 # sum for u' in U [simil(u,i')*(r_u'i - r_u')]

    # 유사도 top N 사용자에 대해 계산
    for user, similarity in nearest[:num_sim_user_top_N]:
      
      # 사용자가 아이템을 평가하지 않았으면 계산하지 않는다.
      if item not in rating_data[user]:
        continue

      sum_u += similarity * (rating_data[user][item] - user_average_rating[user])
      sum_k += abs(similarity)


      # 아이템의 예측 평점 (r_ui)
      # 유사도  Top n 사용자가 모두 아이템을 평가하지 않았으면, 이 아이템에 대해 예측할 수 없음 (0으로 정의한다)
      r_ui = user_average_rating[query_user] + (1/sum_k) * sum_u if sum_k > 0.0 else 0.0

      predicted_rating += [(item,r_ui)] # 예측 평점 list에 저장

    predicted_rating.sort(key=lambda x:x[1],reverse=True)
    print(' '.join(str(item_rating[0]) for item_rating in predicted_rating[:num_item_rec_top_M]))  # 추천 리스트 출력


1. user-item 행렬 데이터 만들기

2. user-user 간의 코사인 유사도로 user 유사도 산출

3. user가 구매하지 않은 item중에서, user간 유사도를 반영한 예측 점수 계산하기

4. 예측 점수가 가장 높은 순 으로 아이템 추천

In [2]:
import math

def ndcg(gt,rec):
    idcg = sum([1.0/math.log(i+2,2) for i in range(len(gt))])
    dcg = 0.0
    for i,r in enumerate(rec):
        if r not in gt:
            continue
        gt_index = gt.index(r)
        if i != gt_index:
            rel = 0.7
        else:
            rel = 1.0
        dcg += rel/math.log(i+2,2)
    ndcg = dcg/idcg

    return ndcg

In [ ]:


insert_list=[]
for idx in range(1,num_users+1):
  for jdx in range(1,num_items+1):
    insert_dic = {'user_id':idx,'item_id':jdx}
    insert_list.append(insert_dic)

base_df = pd.DataFrame(insert_list)
base_df

In [20]:
num_sim_user_top_N = int(input())
num_item_rec_top_M = int(input())

num_users = int(input())
num_items = int(input())
num_rows = int(input())
num_row = []

# 1. user, item 데이터 프레임 만들기
# 2. 그 데이터 프레임에 값 채워넣기

# 데이터 채우기
for _ in range(num_rows):
    temp = input().split(' ')
    
    num_row.append({'user_id':int(temp[0]),'item_id':int(temp[1]),'rating':float(temp[2])})

    # 유저당 10개의 아이템을 할당함.

num_row_df = pd.DataFrame(num_row)



insert_list=[]
for idx in range(1,num_users+1):
  for jdx in range(1,num_items+1):
    insert_dic = {'user_id':idx,'item_id':jdx}
    insert_list.append(insert_dic)

base_df = pd.DataFrame(insert_list)


merged_df = pd.merge(base_df,num_row_df,on=['user_id','item_id'],how='left')

merged_df.fillna(0,inplace=True)


pivot_matrix = merged_df.pivot_table('rating',index='user_id',columns='item_id').fillna(0)
cosine_matrix = cosine_similarity(pivot_matrix,pivot_matrix)
user_sim_matrix = pd.DataFrame(data=cosine_matrix,index=pivot_matrix.index,columns=pivot_matrix.index)

# for idx in range()
# user_sim_matrix[].sort_values(ascending=False)[1:num_sim_user_top_N+1]


num_reco_users = int(input())


# ToDo. 예측 평점 구하기.



# recommend_items= []
# for num_reco_user  in range(1,num_sim_user_top_N+1):
#     sim_users=  user_sim_matrix[num_reco_user].sort_values(ascending=False)[1:num_sim_user_top_N+1].index.to_list()
#     for sim_user in sim_users:
#         recommend_item = pivot_matrix[sim_user].sort_values(ascending=False)[:num_item_rec_top_M].index.to_list()

#         # 사용자가 기존에 평가한 아이템 리스트를 구함
#         check_duplicate = pivot_matrix[num_reco_user].index.to_list()

#         # 추천 결과를 만들 때는 사용가가 기존에 평가한 아이템은 추천 결과에서 제외시켜야한다.
#         checked_recommend_item = list(set(recommend_item) - set(check_duplicate))


#     recommend_items.append(checked_recommend_item)

# recommend_items_1d = [ item for sublist in recommend_items for item in sublist]
# recommend_items_1d




# print('num_item_rec_top_M items for user 1',)
# print('num_item_rec_top_M items for user 2',)


In [25]:
num_row_df

,user_id,item_id,rating
0,1,1,1.0
1,1,2,2.0
2,1,5,1.2
3,2,2,1.5
4,2,3,3.0
5,3,1,2.2
6,3,2,6.2
7,3,7,1.5
8,4,6,1.2
9,4,3,1.5


In [31]:
merged_df

,user_id,item_id,rating
0,1,1,1.0
1,1,2,2.0
2,1,3,NaN
3,1,4,NaN
4,1,5,1.2
5,1,6,NaN
6,1,7,NaN
7,1,8,NaN
8,1,9,NaN
9,1,10,NaN


In [37]:
merged_df.fillna(0,inplace=True)
pivot_matrix = merged_df.pivot_table('rating',index='user_id',columns='item_id')
cosine_matrix = cosine_similarity(pivot_matrix,pivot_matrix)
user_sim_matrix = pd.DataFrame(data=cosine_matrix,index=pivot_matrix.index,columns=pivot_matrix.index)

In [38]:
pivot_matrix

item_id,1,2,3,4,5,6,7,8,9,10
user_id,,,,,,,,,,
1,1.0,2.0,0.0,0.0,1.2,0.0,0.0,0.0,0.0,0.0
2,0.0,1.5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.2,6.2,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0
4,3.1,4.0,1.5,0.0,0.0,1.2,0.0,0.0,0.0,0.0
5,0.0,6.5,0.0,8.2,0.0,0.0,8.0,0.0,0.0,0.0


In [39]:
cosine_matrix

array([[1.        , 0.35245369, 0.85263096, 0.80806489, 0.38892261],
       [0.35245369, 1.        , 0.41092055, 0.57833467, 0.22069431],
       [0.85263096, 0.41092055, 1.        , 0.86572355, 0.58845823],
       [0.80806489, 0.57833467, 0.86572355, 1.        , 0.36467191],
       [0.38892261, 0.22069431, 0.58845823, 0.36467191, 1.        ]])

In [40]:
user_sim_matrix

user_id,1,2,3,4,5
user_id,,,,,
1,1.000000,0.352454,0.852631,0.808065,0.388923
2,0.352454,1.000000,0.410921,0.578335,0.220694
3,0.852631,0.410921,1.000000,0.865724,0.588458
4,0.808065,0.578335,0.865724,1.000000,0.364672
5,0.388923,0.220694,0.588458,0.364672,1.000000


In [84]:
merged_df

,user_id,item_id,rating
0,1,1,1.0
1,1,2,2.0
2,1,3,0.0
3,1,4,0.0
4,1,5,1.2
5,1,6,0.0
6,1,7,0.0
7,1,8,0.0
8,1,9,0.0
9,1,10,0.0


In [83]:
merged_df.loc[(merged_df['user_id']=='target_user')&(merged_df['item_id']=='target_item'),'rating']=result

In [73]:
# 유저 1의 아이템 3에대한 예측 평점을 구한다면?

# 1. 유저  1과 다른 유저들의 유사도를 구한 리스트를 만듬
# 2. 다른 유저가 평가한 아이템 3에 대한 값의 리스트를 만든다.

target_user = 1
target_item = 3

# other_users = list(user_sim_matrix[target_user].sort_values(ascending=False)[1:].values)

other_user_sim_score= user_sim_matrix[target_user].sort_values(ascending=False)[1:].to_dict()

other_users=[]
for key, value in other_user_sim_score.items():
  rating = pivot_matrix[target_item][key]
  temp = {'other_user':key,'sim_score':value,'score_of_other_user':rating}
  other_users.append(temp)


other_users

denominator =[elm['sim_score'] for elm in other_users]

numerator =[elm['sim_score'] * elm['score_of_other_user'] for elm in other_users]


denominator= sum(denominator)

numerator = sum(numerator)

result =  numerator/denominator




[{'other_user': 3,
  'sim_score': 0.8526309573032185,
  'score_of_other_user': 0.0},
 {'other_user': 4,
  'sim_score': 0.8080648855785723,
  'score_of_other_user': 1.5},
 {'other_user': 5,
  'sim_score': 0.3889226131719696,
  'score_of_other_user': 0.0},
 {'other_user': 2,
  'sim_score': 0.3524536884251207,
  'score_of_other_user': 3.0}]

In [79]:
A =[elm['sim_score'] for elm in other_users]

B =[elm['sim_score'] * elm['score_of_other_user'] for elm in other_users]


A = sum(A)

B = sum(B)

C = B/A
C

0.9447919367698965

예측평점구하기

In [90]:
# pivot_matrix
cosine_matrix
# user_sim_matrix

array([[1.        , 0.35245369, 0.85263096, 0.80806489, 0.38892261],
       [0.35245369, 1.        , 0.41092055, 0.57833467, 0.22069431],
       [0.85263096, 0.41092055, 1.        , 0.86572355, 0.58845823],
       [0.80806489, 0.57833467, 0.86572355, 1.        , 0.36467191],
       [0.38892261, 0.22069431, 0.58845823, 0.36467191, 1.        ]])

In [93]:
cosine_matrix[0][0]

sim_sum = sum(cosine_matrix[0])
sim_sum



3.402072144478881

In [ ]:
for idx in range(pivot_matrix.shape[0]):
    for jdx in range(pivot_matrix.shape[1]):
        if pivot_matrix[idx][jdx] == 0.0:
            

In [52]:
user_sim_matrix = pd.DataFrame(data=cosine_matrix,index=pivot_matrix.index,columns=pivot_matrix.index)

num_reco_users =2 , 1, 2 명에 대한 각각 2명의 유사도가 높은 사용자를 뽑고

num_item_rec_top_M개 만큼의 아이템을 뽑음

In [69]:

# 입력을 받지 않은 값, 0으로 채운다음에 해당 코드 적재하기
recommend_items= []
for num_reco_user  in range(1,num_sim_user_top_N+1):
    sim_users=  user_sim_matrix[num_reco_user].sort_values(ascending=False)[1:num_sim_user_top_N+1].index.to_list()
    for sim_user in sim_users:
        recommend_item = pivot_matrix[sim_user].sort_values(ascending=False)[:num_item_rec_top_M].index.to_list()

        check_duplicate = pivot_matrix[sim_user].index.to_list()

        # 중복 제거
        checked_recommend_item = list(set(recommend_item) - set(check_duplicate))


    recommend_items.append(checked_recommend_item)

recommend_items_1d = [ item for sublist in recommend_items for item in sublist]
recommend_items_1d

[5, 1, 2, 4]

In [12]:
num_rows = int(input())
num_row = []
for _ in range(num_rows):
    temp = input().split(' ')

    num_row.append({'user_id':int(temp[0]),'item_id':int(temp[1]),'rating':float(temp[2])})

print(num_row)

num_row_df = pd.DataFrame(num_row)
num_row_df

[{'user_id': 1, 'item_id': 1, 'rating': 2.34}, {'user_id': 1, 'item_id': 2, 'rating': 6.21}]


,user_id,item_id,rating
0,1,1,2.34
1,1,2,6.21


In [32]:
num_row_df
pivot_table = num_row_df.pivot_table('rating',index='user_id',columns='item_id').fillna(0)
pivot_table

item_id,1,2
user_id,,
1,2.34,6.21


In [42]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_df = cosine_similarity(pivot_table,pivot_table)

user_sim_df = pd.DataFrame(data=cosine_df,index=pivot_table.index,columns=pivot_table.index)


recommend_list= []

for idx in range(num_reco_users):
    recommend = user_sim_df[idx].sort_values(ascending=False)[1:num_sim_user_top_N+1]   
    recommend_list.append(recommend)

print(recommend_list)

NameError: name 'num_reco_users' is not defined

In [15]:
df1_list =[{'user_id':1,'item_id':1,'rating':2.1},{'user_id':1,'item_id':2,'rating':2.2},{'user_id':1,'item_id':3,'rating':2.3}]

df1 = pd.DataFrame(df1_list)
df1

,user_id,item_id,rating
0,1,1,2.1
1,1,2,2.2
2,1,3,2.3


In [19]:
merge_df = pd.merge(base_df,df1,on=['user_id','item_id'],how='left')
merge_df

,user_id,item_id,rating
0,1,1,2.1
1,1,2,2.2
2,1,3,2.3
3,1,4,NaN
4,1,5,NaN
5,2,1,NaN
6,2,2,NaN
7,2,3,NaN
8,2,4,NaN
9,2,5,NaN


In [4]:
df2_list =[{'id':1,'item':1,'rating':2.1},{'id':1,'item':2,'rating':2.2},{'id':1,'item':3,'rating':2.3},{'id':1,'item':4,'rating':0},{'id':1,'item':5,'rating':0}]
df2 = pd.DataFrame(df2_list)
df2

,id,item,rating
0,1,1,2.1
1,1,2,2.2
2,1,3,2.3
3,1,4,0.0
4,1,5,0.0


In [5]:
df_merge = pd.merge(df2,df1,on=['id','item'],how='left')
df_merge

,id,item,rating_x,rating_y
0,1,1,2.1,2.1
1,1,2,2.2,2.2
2,1,3,2.3,2.3
3,1,4,0.0,NaN
4,1,5,0.0,NaN
